In [1]:
import os
from googlesearch import search
import pandas as pd
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests import get
import json
from bs4 import BeautifulSoup
from selenium_recaptcha import Recaptcha_Solver
from selenium.webdriver.chrome.options import Options

import time



In [2]:
path = "../Documents/Ticket Sales.xlsx"
sales = pd.read_excel(path, sheet_name ="Events")


In [3]:



options =  webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\eric9\\AppData\\Local\\Google\\Chrome\\User Data\Default")

# service = webdriver.ChromeService(executable_path=r'C:\Program Files\Google\Chrome\Application\chrome.exe')
# service = webdriver.ChromeService(executable_path=r'C:\Users\eric9\Downloads\chromedriver_win32.zip\chromedriver.exe')
# service = webdriver.ChromeService()
driver = webdriver.Chrome(options=options)

In [4]:
def convert_string_to_number(s):
    s = s.lower().strip()
    # Handle comma-separated numbers
    if ',' in s:
        return int(s.replace(',', ''))
    # Handle suffixes like K, M, B
    elif 'k' in s:
        return int(float(s.replace('k', '')) * 1000)
    elif 'm' in s:
        return int(float(s.replace('m', '')) * 1000000)
    elif 'b' in s:
        return int(float(s.replace('b', '')) * 1000000000)
    else:
        return int(s)
    

In [5]:
def solver():
    try:
        solver = Recaptcha_Solver(
        driver=driver, # Your Web Driver
        ffmpeg_path='', # Optional. If does not exists, it will automatically download.
        log=1 # If you want to view the progress.
        )
        solver.solve_recaptcha()
    #     driver.switch_to.default_content()

    #     driver.find_element(By.XPATH, ".//iframe[@title='reCAPTCHA']")
    #     driver.switch_to.frame(driver.find_element(By.XPATH, ".//iframe[@title='reCAPTCHA']"))
    #     if len(driver.find_elements(By.XPATH, "//*[@id='recaptcha-label']")) > 0:
    #         driver.find_element(By.XPATH, "//*[@id='recaptcha-label']").click()
    #     if len(driver.find_elements(By.XPATH, "//*[@id='recaptcha-anchor']")) > 0 :
    #         if driver.find_element(By.XPATH, "//*[@id='recaptcha-anchor']").get_attribute("aria-checked") !="true" :  
    #             solver.request_audio_version()
    #             solver.solve_audio_captcha()
    except:
        pass

In [6]:
# Perform a Google search using Selenium
def perform_google_search(query):
    driver.get(f"https://www.google.com/search?q={query}")
    try:
        driver.find_element(By.XPATH, ".//iframe[@title='reCAPTCHA']")
        solver()
        time.sleep(2)
    except:
        pass

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    first_result = soup.find('div', class_='g')
    if first_result:
        first_link = first_result.find('a')['href']
        return first_link
    else:
        return list(search(query, advanced= True, num_results = 1))[0].url




In [7]:
perform_google_search("instagram profile " + "illenium")

'https://www.instagram.com/illenium/?hl=en'

## Instagram

In [8]:


def get_insta_followers(s,username = ""):
    if username == "":  
        url = perform_google_search(("instagram profile " + s))
        username = re.search(r'instagram\.com/([^/?]+)', url).group(1)
    else:
        url = "https://www.instagram.com/" + username
    page = get(url, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = soup.find_all('meta', attrs={'property': 'og:description'})
    if len(data) == 0:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = soup.find_all('meta', attrs={'property': 'og:description'})

    follower_count = data[0].get('content').split()[0]    
    return convert_string_to_number(follower_count)

In [9]:
get_insta_followers("illenium",)

859000

## Twitter

In [10]:


def get_twitter_followers(s,username = ""):
    if username == "":
        url = perform_google_search("twitter profile " + s)
        username = re.search(r'twitter\.com/([^/?]+)', url).group(1)
    else:
        url = "https://www.x.com/" + username
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Wait for the follower element to be present
    followers_xpath = '//a[contains(@href, "/verified_followers") and @dir="ltr" and @role="link"]'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, followers_xpath)))
    

    follower_element = driver.find_element(By.XPATH, followers_xpath)
    follower_element_text = follower_element.text.replace(' Followers', '')
    follower_count = convert_string_to_number(follower_element_text)
    return follower_count


## Spotify

In [4]:
from http import client
import requests
import json


with open("../spotify_credentials.json", "r") as f:
    credentials = json.load(f)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]
auth_url = credentials["auth_url"]
response = requests.post(
    auth_url,
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
)

# Check if the request was successful
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print("Access token:", access_token)
else:
    print("Failed to retrieve access token:", response.status_code, response.text)

Access token: BQDTO0fJptJQYbwj6TjHB77pqeUfkxEo5Fa-gPR62CUi5acwgponlP1bf2ehMvab920BhsV_EdSyYqmskaeuL3Iji0dIFhMlvk-efbdIAUNifdwKeYE


In [17]:
headers = {
    "Authorization": f"Bearer {access_token}"
}

from math import nan
import requests
def get_spot_stats(artist):
    search_url = f"https://api.spotify.com/v1/search?q=artist:{artist}&type=artist&limit=1"
    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        search_results = response.json()
        # print(json.dumps(search_results, indent=4))
        if len(search_results['artists']['items']) == 0:
            # print("No search results found")
            return  {
            'genre': pd.NA,
            'followers': 0,
            'popularity': 0,
            'listens': 0
        }
        genre = search_results['artists']['items'][0]['genres'][0]
        followers  = search_results['artists']['items'][0]['followers']['total']
        popularity = search_results['artists']['items'][0]['popularity']
        url = search_results['artists']['items'][0]['external_urls']['spotify']
        response = requests.get(url,)
        soup = BeautifulSoup(response.content, 'html.parser')
        meta_tag = soup.find('meta', attrs={'property': 'og:description'})
        if meta_tag:
            content = meta_tag.get('content')
            match = re.search(r'(\d+(\.\d+)?[MK]?) monthly listeners', content)
            if match:
                listens = convert_string_to_number(match.group(1))
            else :
                listens = 0
        return {
            'genre': genre,
            'followers': followers,
            'popularity': popularity,
            'listens': listens
        }
        return  
    else:
        print("Failed to retrieve search results:", response.status_code, response.text)
        return {
            'genre': pd.NA,
            'followers': 0,
            'popularity': 0,
            'listens': 0
        }
    
get_spot_stats("illenium")

{'genre': 'edm', 'followers': 1538144, 'popularity': 72, 'listens': 8300000}

In [23]:
# import requests

# search_query = "genre:edm"
# search_type = "artist"
# search_url = f"https://api.spotify.com/v1/search?q={search_query}&type={search_type}"

# response = requests.get(search_url, headers=headers)

# if response.status_code == 200:
#     search_results = response.json()
#     # print(json.dumps(search_results, indent=4))
# else:
#     print("Failed to retrieve search results:", response.status_code, response.text)
# # print(search_results['artists']['items'][0]['followers']['total'])
# # print(search_results['artists']['items'][0]['popularity'])
# # Extract relevant information from search results
# artists = search_results['artists']['items']
# data = []
# for artist in artists:
#     data.append({
#         'name': artist['name'],
#         'followers': artist['followers']['total'],
#         'popularity': artist['popularity'],
#         'genres': ', '.join(artist['genres']),
#         'spotify_url': artist['external_urls']['spotify']
#     })


# # Create a DataFrame from the extracted data
# df_artists = pd.DataFrame(data)
# df_artists = df_artists.sort_values(by='popularity', ascending=False)
# print(df_artists)

In [13]:
from torch import res


def get_stubhub_favourites(artist):
    try:
        pageSource = requests.get("https://www.stubhub.ca/secure/search?q=" + artist).content
    except:
        pageSource = driver.get("https://www.stubhub.ca/secure/search?q=" + artist).page_source
    soup = BeautifulSoup(pageSource, 'html.parser')
    script_tag = soup.find('script', {'id': 'index-data', 'type': 'application/json'})

    json_data = json.loads(script_tag.string)
    return json_data['topSearchResults'][0]['aggregateFavorites']

    


In [ ]:
# Initialize a list to store the results
results = []

# Iterate through each artist in the sales DataFrame
for artist in sales['Artist'].unique():
    print(artist)
    try:
        # Get Instagram followers
        insta_followers_count = get_insta_followers(artist)

        # Get Twitter followers
        twitter_followers_count = get_twitter_followers(artist)

        # Get Spotify stats
        spotify_stats = get_spot_stats(artist)
        spotify_followers_count = spotify_stats["followers"]
        spotify_popularity_count = spotify_stats["popularity"]
        spotify_monthly_listeners_count = spotify_stats["listens"]
        genre = spotify_stats["genre"]

        # Get Stubhub favourites
        stubhub_favourites_count = get_stubhub_favourites(artist)

        # Store the results in a dictionary
        result = {
            'Artist': artist,
            'Instagram Followers': insta_followers_count,
            'Twitter Followers': twitter_followers_count,
            'Spotify Followers': spotify_followers_count,
            'Spotify Popularity': spotify_popularity_count,
            'Spotify Monthly Listeners': spotify_monthly_listeners_count,
            'Genre': genre,
            'Stubhub Favourites': stubhub_favourites_count
        }
        results.append(result)

        print(f"Instagram: {insta_followers_count}, Twitter: {twitter_followers_count}, Genre: {genre}, Spotify Followers: {spotify_followers_count}, Popularity: {spotify_popularity_count}, Monthly Listeners: {spotify_monthly_listeners_count}, Stubhub: {stubhub_favourites_count}")

    except Exception as e:
        print(f"An error occurred for artist {artist}: {e}")



$uicide Boy$
Recaptcha Solvedbe...


In [26]:
results_df = pd.DataFrame(results)


In [18]:
# Initialize a list to store the results
results2 = []
results_df = pd.read_excel(path, sheet_name="Socials")
# Iterate through each artist in the sales DataFrame
for artist in sales['Artist'].unique():
    if artist not in results_df['Artist'].values:
        print(artist)
        try:
            # Get Instagram followers
            insta_followers_count = get_insta_followers(artist)

            # Get Twitter followers
            twitter_followers_count = get_twitter_followers(artist)

            # Get Spotify stats
            spotify_stats = get_spot_stats(artist)
            spotify_followers_count = spotify_stats["followers"]
            spotify_popularity_count = spotify_stats["popularity"]
            spotify_monthly_listeners_count = spotify_stats["listens"]
            genre = spotify_stats["genre"]

            # Get Stubhub favourites
            stubhub_favourites_count = get_stubhub_favourites(artist)

            # Store the results in a dictionary
            result = {
                'Artist': artist,
                'Instagram Followers': insta_followers_count,
                'Twitter Followers': twitter_followers_count,
                'Spotify Followers': spotify_followers_count,
                'Spotify Popularity': spotify_popularity_count,
                'Spotify Monthly Listeners': spotify_monthly_listeners_count,
                'Genre': genre,
                'Stubhub Favourites': stubhub_favourites_count
            }
            results2.append(result)

            print(f"Instagram: {insta_followers_count}, Twitter: {twitter_followers_count}, Genre: {genre}, Spotify Followers: {spotify_followers_count}, Popularity: {spotify_popularity_count}, Monthly Listeners: {spotify_monthly_listeners_count}, Stubhub: {stubhub_favourites_count}")

        except Exception as e:
            print(f"An error occurred for artist {artist}: {e}")



Artemas
An error occurred for artist Artemas: list index out of range
Chelsea Cutler and Jeremy Zucker
Instagram: 390000, Twitter: 110400, Genre: <NA>, Spotify Followers: 0, Popularity: 0, Monthly Listeners: 0, Stubhub: 35
Danny Avila
Recaptcha Solvedbe...
An error occurred for artist Danny Avila: list index out of range
Day 1 DPRTMNT AP
An error occurred for artist Day 1 DPRTMNT AP: 'NoneType' object has no attribute 'group'
Day 1 Rebel AP
Instagram: 85000, Twitter: 2696, Genre: <NA>, Spotify Followers: 0, Popularity: 0, Monthly Listeners: 0, Stubhub: 0
Day 3 Rebel AP
Instagram: 85000, Twitter: 2696, Genre: <NA>, Spotify Followers: 0, Popularity: 0, Monthly Listeners: 0, Stubhub: 0
Electric Island Afterparty Coda
Instagram: 156, Twitter: 2226, Genre: <NA>, Spotify Followers: 0, Popularity: 0, Monthly Listeners: 0, Stubhub: 0
Emorfik
An error occurred for artist Emorfik: list index out of range
Frank Walker
An error occurred for artist Frank Walker: list index out of range
IsoKnock
An 

In [19]:
results_df2 = pd.DataFrame(results2)


In [20]:
results_df = pd.concat([results_df, results_df2])

In [21]:
with pd.ExcelWriter(path,  mode='a',  engine="openpyxl", if_sheet_exists="replace") as writer:
    results_df.to_excel(writer, sheet_name = "Socials", index = False)